<a href="https://colab.research.google.com/github/divyansh212/AI-MODELS-/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
import zipfile
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
zip_path = "/content/archive.zip"
extract_path = "/content/archive"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extracted to:", extract_path)
torch.manual_seed(42)
df = pd.read_csv('/content/archive/fashion-mnist_train.csv')
x=df.iloc[:,1:].values
y=df.iloc[:,0].values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train=x_train/255.0
x_test=x_test/255.0
class customdataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.labels=torch.tensor(labels,dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self, index):
    return self.features[index],self.labels[index]
training_dataset=customdataset(x_train,y_train)
test_dataset=customdataset(x_test,y_test)
train_loader=DataLoader(training_dataset,batch_size=32,shuffle=True)
test_Loader=DataLoader(test_dataset,batch_size=32,shuffle=False)
class neural_network(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.model=nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
    )
  def forward(self,x):
    return self.model(x)
epoch=100
lr=0.01
model=neural_network(x_train.shape[1])
model=model.to(device)
c=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=lr)
for i in range(epoch):
  total_epoch_loss=0
  for batch_features,batch_labels in train_loader:
    batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)
    output=model(batch_features)
    loss=c(output,batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_epoch_loss+=loss.item()
  avg_loss=total_epoch_loss/len(train_loader)
model.eval()
total=0
correct=0
with torch.no_grad():
  for batch_features,batch_labels in test_Loader:
    batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)
    output=model(batch_features)
    _,predicted=torch.max(output,1)
    total+=batch_labels.shape[0]
    correct+=(predicted==batch_labels).sum().item()
print(correct/total)


Extracted to: /content/archive
0.8859166666666667
